In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.primitives import Sampler
from qiskit.visualization import plot_bloch_vector, plot_histogram
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from qiskit.quantum_info import Statevector
from mpl_toolkits.mplot3d import Axes3D

def create_custom_state(theta, phi):
    qc = QuantumCircuit(1)
    qc.ry(theta, 0)
    qc.rz(phi, 0)
    return qc

def bloch_coordinates(statevector):
    x = 2 * np.real(np.conj(statevector[0]) * statevector[1])
    y = 2 * np.imag(np.conj(statevector[0]) * statevector[1])
    z = np.real(np.conj(statevector[0]) * statevector[0] - np.conj(statevector[1]) * statevector[1])
    return [x, y, z]

def simulate_and_visualize(theta, phi):
    # Create the circuit
    qc = create_custom_state(theta, phi)
    
    # Create a measurement circuit
    meas_qc = QuantumCircuit(1, 1)
    meas_qc.measure(0, 0)
    
    # Combine circuits
    complete_qc = qc.compose(meas_qc)
    
    # Simulate the circuit
    sampler = Sampler()
    job = sampler.run(complete_qc, shots=1000)
    result = job.result()
    counts = result.quasi_dists[0].binary_probabilities()
    
    # Get statevector for Bloch sphere
    statevector = Statevector.from_instruction(qc)
    
    # Calculate Bloch vector coordinates
    bloch_vector = bloch_coordinates(statevector)

    # Create subplots
    fig = plt.figure(figsize=(18, 6))
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132, projection='3d')
    ax3 = fig.add_subplot(133)
    
    # Plot circuit
    complete_qc.draw('mpl', ax=ax1)
    ax1.set_title('Quantum Circuit')
    
    # Plot Bloch sphere
    plot_bloch_vector(bloch_vector, ax=ax2)
    ax2.set_title('Bloch Sphere')
    
    # Plot histogram
    plot_histogram(counts, ax=ax3)
    ax3.set_title('Measurement Outcome')
    
    plt.tight_layout()

    # Make Bloch sphere rotatable test
    def on_motion(event):
        if event.inaxes == ax2:
            ax2.view_init(elev=event.ydata, azim=event.xdata)
            fig.canvas.draw_idle()

    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    
    plt.show()

# Create interactive widget
interact(simulate_and_visualize,
         theta=FloatSlider(min=0, max=2*np.pi, step=0.1, description='θ:'),
         phi=FloatSlider(min=0, max=2*np.pi, step=0.1, description='φ:'))

interactive(children=(FloatSlider(value=0.0, description='θ:', max=6.283185307179586), FloatSlider(value=0.0, …

<function __main__.simulate_and_visualize(theta, phi)>